# Lang2LTL version 2

## Prerequisite: Import libraries 

In [ ]:
import pandas as pd
import json
import random
import os

from rer import *

## 1. Generate natural language commands/utterances

### Load samples for utterances

In [ ]:
# -- load samples of LTL utterances from a CSV file using Pandas library:
ltl_samples = pd.read_csv(open('data/symbolic_batch12_noperm_pun.csv', 'r'))
display(ltl_samples[:10])

city_jsons = os.listdir('osm/')
selected_city = random.choice(city_jsons)
display(selected_city)

# -- load a city's landmarks from a JSON file (obtained from OpenStreetMap):
landmarks = json.load(open(f'osm/{selected_city}', 'r', encoding='utf8'))
display(list(landmarks.keys()))

# NOTE: several prepositions are taken from Kaiyu's paper (https://h2r.cs.brown.edu/wp-content/uploads/zheng2021spatial.pdf)
# -- each key is the phrase describing the spatial relation
# -- a key's corresponding value indicates the number of arguments it would take
spatial_relations = [
    {'between': 2},
    {'in front of': 1},
    {'behind': 1},
    {'next to': 1},
    {'adjacent to': 1},
    {'opposite': 1},
    {'near': 1},
    {'left of': 1},
    {'right of': 1},
    {'at': 1},
    {'north of': 1},
    {'south of': 1},
    {'east of': 1},
    {'west of': 1},
]

# -- list of landmarks in the city
landmarks = list(landmarks.keys())

# -- list of objects to use as reference objects:
generic_city_objects = [
    'bench', 'statue', 'stairs', 'store', 'door', 'bus stop', 'gas station', 'coffee shop', 
    'bicycle rack', 'parking meter', 'parking spot', 'parking space', 'stop sign', 
]


In [ ]:
# NOTE: this is Spot experiment specific content:

selected_city = 'blackstone.json'

# -- load a city's landmarks from a JSON file (obtained from OpenStreetMap):
landmarks = json.load(open(f'osm/{selected_city}', 'r', encoding='utf8'))
all_landmarks = list(landmarks.keys())

landmarks_to_use = ['Wildflour', 'Garden Grille Cafe']
for L in all_landmarks:
    if L not in landmarks_to_use:
        landmarks.pop(L)

# -- transforming specific names of large landmarks into generic categorical names:
landmarks['bakery'] = landmarks.pop('Wildflour')
landmarks['restaurant'] = landmarks.pop('Garden Grille Cafe')

# NOTE: below are the objects found in experiment location:
generic_city_objects = ['bicycle rack', 'chair', 'car']

landmarks = list(landmarks.keys())

### Define function for generating utterances

In [ ]:
generation_params = {
	'samples': ltl_samples, 
	'landmarks': landmarks, 
	'city_objects': generic_city_objects,
	'spatial_relations': spatial_relations
}

num_commands = 4

# -- forcing minimum number of propositions to be 5, forcing all SREs for propositions:
used_templates, utterances = generate_synthetic_command(generation_params, num_utterances=num_commands, min_props=2, force_sre=True)

for N in range(num_commands):
	print(f'{utterances[N]}\n')

## 2. Prompt LLM for spatial referring expressions

In [ ]:
count = 0

for command in utterances:
    raw_output, parsed_output = referring_exp_recognition(command)

    parsed_output['lifted_gtr'] = used_templates[count]

    count += 1

    print(f'Command: {command}\n{json.dumps(parsed_output,indent=2)}')
    # display(parsed_output['command'])
    print(f"Groundtruth matched?: {parsed_output['lifted_llm'] == parsed_output['lifted_gtr']}" )
    print(f"-> LLM Lifted Translation: {parsed_output['lifted_llm']}")
    print(f"-> Groundtruth Lifted Translation: {parsed_output['lifted_gtr']}")


In [ ]:
utterances = list(filter(None, [X.strip() for X in open('blackstone_commands.txt', 'r').readlines()]))

print(utterances)

count = 0

all_llm_output = []

for command in utterances:
    raw_output, parsed_output = referring_exp_recognition(command)

    # parsed_output['lifted_gtr'] = used_templates[count]

    count += 1

    print(f'Command: {command}\n{json.dumps(parsed_output,indent=2)}')
    # display(parsed_output['command'])
    
    all_llm_output.append(parsed_output)
    
json.dump(all_llm_output, open('blackstone_srer_output.json', 'w'))